# Modeling

Spencer Brothers

The next step is to explore different modeling ideas for the project, with the aim of developing a model that beats a benchmark model (such as the majority class classifier) and produces results that -- hopefully! -- can be used to solve the business problem.

## Objectives

- Practice feature engineering to improve model performance.

- Practice cross-validation.

- Learn about the properties of different modeling algorithms by experimenting with different methods and comparing different candidate models.

- Learn from your group members.

## Tasks

- Set up a training set and a validation set using application_train.csv data set to do cross-validation.  Alternatively you could perform cross-validation using a different framework, such as k-fold cross validation as implemented in modeling packages such as caret or tidymodels or scikit-learn. The model performance that matters, of course, is the estimated performance on the test set as well as the Kaggle score.

- Identify the performance benchmark established by the majority class classifier.

- Fit several different logistic regression models using different predictors. Do interaction terms improve the model?  Compare model performance using not just accuracy but also AUC.
Explore using algorithms like random forest and gradient boosting. Compare model performance.

- Perform the data transformations required by a given algorithm.  For example, some algorithms require numeric data and perform better when it has been standardized or normalized.
Experiment with upsampling and downsampling the data to adjust for the imbalanced target variable.  (See APM Ch. 16.)  Does this strategy this improve model performance?

- Try combining model predictions--this is called an ensemble model--to improve performance.

- Try additional feature engineering to boost model performance. Can you combine variables or bin numeric variables?  Explore the notebooks at Kaggle for data transformation ideas. In particular, use the other data sets at Kaggle--beyond the application data--to create additional features.

- For machine learning models experiment with hyperparameter tuning  to try to boost performance.


In [ ]:
# Regression Section

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# Load data
train_data = pd.read_csv('application_train.csv')
bureau_data = pd.read_csv('bureau.csv')
previous_app_data = pd.read_csv('previous_application.csv')

# Aggregate bureau data
bureau_agg = bureau_data.groupby('SK_ID_CURR').agg({
    'AMT_CREDIT_SUM': ['sum', 'mean'],
    'AMT_CREDIT_SUM_DEBT': ['sum', 'mean'],
    'AMT_CREDIT_SUM_OVERDUE': ['sum', 'mean'],
    'CNT_CREDIT_PROLONG': ['sum']
})
bureau_agg.columns = ['_'.join(col).upper() for col in bureau_agg.columns]
bureau_agg.reset_index(inplace=True)

# Aggregate previous application data
previous_app_agg = previous_app_data.groupby('SK_ID_CURR').agg({
    'AMT_APPLICATION': ['mean', 'max'],
    'AMT_CREDIT': ['mean', 'max'],
    'DAYS_DECISION': ['mean'],
    'NAME_CONTRACT_STATUS': ['count']
})
previous_app_agg.columns = ['_'.join(col).upper() for col in previous_app_agg.columns]
previous_app_agg.reset_index(inplace=True)

# Merge aggregated features with training data
train_data = train_data.merge(bureau_agg, on='SK_ID_CURR', how='left')
train_data = train_data.merge(previous_app_agg, on='SK_ID_CURR', how='left')

# Select features and target
features = ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH',
            'AMT_CREDIT_SUM_SUM', 'AMT_CREDIT_SUM_MEAN', 'AMT_CREDIT_SUM_DEBT_SUM', 'AMT_CREDIT_SUM_DEBT_MEAN',
            'AMT_APPLICATION_MEAN', 'AMT_APPLICATION_MAX', 'AMT_CREDIT_MEAN', 'AMT_CREDIT_MAX']
target = 'TARGET'

X = train_data[features]
y = train_data[target]

# Handle missing values (simple imputation)
X.fillna(X.median(), inplace=True)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Baseline model: Majority class classifier (predicts the most frequent class)
majority_class = y_train.mode()[0]
y_majority_pred = np.full_like(y_val, majority_class)
majority_accuracy = accuracy_score(y_val, y_majority_pred)
print(f'Majority Class Accuracy: {majority_accuracy:.4f}')

# Perform k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Fit logistic regression model with cross-validation
model = LogisticRegression()
cross_val_auc = cross_val_score(model, X_train_scaled, y_train, cv=kf, scoring='roc_auc')
print(f'Cross-Validation AUC: {cross_val_auc.mean():.4f}')

# Train logistic regression model on full training set
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_val_scaled)
y_prob = model.predict_proba(X_val_scaled)[:, 1]

# Evaluate model
accuracy = accuracy_score(y_val, y_pred)
roc_auc = roc_auc_score(y_val, y_prob)

print(f'Validation Accuracy: {accuracy:.4f}')
print(f'Validation ROC AUC Score: {roc_auc:.4f}')

# Fit logistic regression models with interaction terms
X_train['INCOME_CREDIT_RATIO'] = X_train['AMT_INCOME_TOTAL'] / X_train['AMT_CREDIT']
X_val['INCOME_CREDIT_RATIO'] = X_val['AMT_INCOME_TOTAL'] / X_val['AMT_CREDIT']

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

model_interaction = LogisticRegression()
model_interaction.fit(X_train_scaled, y_train)

# Make predictions
y_pred_interaction = model_interaction.predict(X_val_scaled)
y_prob_interaction = model_interaction.predict_proba(X_val_scaled)[:, 1]

# Evaluate interaction model
accuracy_interaction = accuracy_score(y_val, y_pred_interaction)
roc_auc_interaction = roc_auc_score(y_val, y_prob_interaction)

print(f'Validation Accuracy with Interaction: {accuracy_interaction:.4f}')
print(f'Validation ROC AUC Score with Interaction: {roc_auc_interaction:.4f}')


<ipython-input-3-d7345e4c8f1b>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(X.median(), inplace=True)


Majority Class Accuracy: 0.9193
Cross-Validation AUC: 0.6251
Validation Accuracy: 0.9192
Validation ROC AUC Score: 0.6218
Validation Accuracy with Interaction: 0.9192
Validation ROC AUC Score with Interaction: 0.6243


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
